In [1]:
import re
import jieba

In [2]:
def strQ2B(ustring):
    """全角转半角"""
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        if inside_code == 12288:                              #全角空格直接转换            
            inside_code = 32 
        elif (inside_code >= 65281 and inside_code <= 65374): #全角字符（除空格）根据关系转化
            inside_code -= 65248

        rstring += chr(inside_code)
    return rstring

In [3]:
def clean(text,vocab):
    text=strQ2B(text)
    text=re.sub(r'\ue40c','',text)
    text=re.sub('[:「」￥…，,(【嘻嘻】)【哈哈】;"”“+/—!. _ - % \[\]*◎《》、。]', '', text)  # 去特殊字符
    text=re.sub('(www\.(.*?)\.com)|(http://(.*?)\.com)','',text.lower()) #去URL
    text=re.sub('[a-zA-Z]+',' EN ',text) # 去英文
    text=re.sub('([\d]*年*[\d]*月*[\d]+日+)|([\d]+年+)|([\d]*年*[\d]+月+)',' DATE ',text)#去日期
    text=re.sub('[\d]+',' NUMBER ',text) #去数字
    seg_list = jieba.cut(text, cut_all=False)
    text=' '.join(seg_list)
    for word in text.split(' '):
        vocab[word]=vocab.get(word,0)+1
    #结巴分词
    
    return text

In [4]:
def raw_form(path,title_path,content_path,vocab):
    contents=[]
    titles=[]
    with open(path,'r',encoding='ansi')as f:
        text=''.join(f.readlines())
        p=re.compile('<contenttitle>(.*)</contenttitle>\n<content>(.*)</content>')
        for temp in p.finditer(text):
            title=(temp.group(1))
            content=(temp.group(2))
            if title.strip()!='' and content.strip()!='':
                s1=clean(title,vocab)
                if len(s1)>60 :
                    s1=s1[:60]
                s2=clean(content,vocab)
                if len(s2)>240 :
                    s2=s2[:240]
                titles.append(s1)
                contents.append(s2)
    with open(title_path,'w',encoding='utf-8') as wf:
        for temp in titles:
            wf.write(temp+'\n')
    with open(content_path,'w',encoding='utf-8') as wf:
        for temp in contents:
            wf.write(temp+'\n')
    return contents,titles

In [5]:
import os

In [6]:
raw_path="./data_test"
form_path='./train_test'
vocab_path='./vocab/vocab.txt'
max_vocabulary_size=50000

In [7]:
def all_raw_fom(raw_path,form_path,vocab_path,max_vocabulary_size):
    vocab={}
    special_words = ['<PAD>', '<UNK>', '<GO>',  '<EOS>']
    file_list=os.listdir(raw_path)
    print(file_list)
    for item in file_list:
        _,_=raw_form(os.path.join(raw_path,item),os.path.join(form_path,'title_'+item),os.path.join(form_path,'content_'+item),vocab)
    
    vocab_list=special_words+sorted(vocab, key=vocab.get, reverse=True)
    if len(vocab_list)>max_vocabulary_size:
        vocab_list=vocab_list[:max_vocabulary_size]
    with open(vocab_path,'w',encoding='utf-8')as f:
        for w in vocab_list:
            f.write(w+'\n')
    return

In [8]:
all_raw_fom(raw_path,form_path,vocab_path,max_vocabulary_size)


['news1.txt', 'news2.txt']


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jiangpin\AppData\Local\Temp\jieba.cache
Loading model cost 3.124 seconds.
Prefix dict has been built succesfully.


输入字典 生成字典 输出tokenize

In [9]:
def fom_tokenize(form_path,tokenize_path,vocab_path):

    vocab_list=[]
    with open(vocab_path,'r',encoding='utf-8')as f:
        for item in f.readlines():
            vocab_list.append(item.strip())
    int_to_vocab = {idx: word for idx, word in enumerate(vocab_list)}
    vocab_to_int = {word: idx for idx, word in int_to_vocab.items()}
    file_list=os.listdir(form_path)
    print(file_list)
    for item in file_list:
        with open(os.path.join(form_path,item),'r',encoding='utf-8')as f:
#             tokenize=[]
#             for sentence in f.readlines():
#                 sentence_list=sentence.strip().split(' ')
#                 temp=[]
#                 for word in sentence_list:
#                     temp.append()
            if re.compile('title').search(item):
                tokenize=[[vocab_to_int.get(word,vocab_to_int['<UNK>']) for word in sentence.strip().split(' ')]+[vocab_to_int['<EOS>']]for sentence in f.readlines()]
            else:
                tokenize=[[vocab_to_int.get(word,vocab_to_int['<UNK>']) for word in sentence.strip().split(' ')]for sentence in f.readlines()]
            with open(os.path.join(tokenize_path,item),'w',encoding='utf-8')as wf:
                for line in tokenize:
                    s=' '.join([str(w) for w in line])
                    wf.write(s+'\n')

In [12]:
form_path='./train_test'
tokenize_path='./tokenize'
vocab_path='./vocab/vocab.txt'

In [13]:
fom_tokenize(form_path,tokenize_path,vocab_path)

['content_news1.txt', 'content_news2.txt', 'title_news1.txt', 'title_news2.txt']


In [14]:
from distutils.version import LooseVersion
import tensorflow as tf
from tensorflow.python.layers.core import Dense


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.8.0


In [15]:
import numpy as np
import time
import tensorflow as tf
import os

In [16]:
vocab_path='./vocab/vocab.txt'

In [17]:
def get_vocab(vocab_path):
    vocab_list=[]
    with open(vocab_path,'r',encoding='utf-8')as f:
        for item in f.readlines():
            vocab_list.append(item.strip())
    int_to_vocab = {idx: word for idx, word in enumerate(vocab_list)}
    vocab_to_int = {word: idx for idx, word in int_to_vocab.items()}
    return int_to_vocab,vocab_to_int

In [18]:
source_int_to_letter, source_letter_to_int=get_vocab(vocab_path)
target_int_to_letter, target_letter_to_int =source_int_to_letter, source_letter_to_int

In [27]:
len(source_int_to_letter)

50000

构建模型

In [28]:
def get_inputs():
    '''
    模型输入tensor
    '''
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    # 定义target序列最大长度（之后target_sequence_length和source_sequence_length会作为feed_dict的参数）
    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return inputs, targets, learning_rate, target_sequence_length, max_target_sequence_length, source_sequence_length

In [29]:
def get_encoder_layer(input_data, rnn_size, num_layers,
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):

    '''
    构造Encoder层
    
    参数说明：
    - input_data: 输入tensor
    - rnn_size: rnn隐层结点数量
    - num_layers: 堆叠的rnn cell数量
    - source_sequence_length: 源数据的序列长度
    - source_vocab_size: 源数据的词典大小
    - encoding_embedding_size: embedding的大小
    '''
    # Encoder embedding
    encoder_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, encoding_embedding_size)

    # RNN cell
    def get_lstm_cell(rnn_size):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return lstm_cell

    cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size) for _ in range(num_layers)])
    
    encoder_output, encoder_state = tf.nn.dynamic_rnn(cell, encoder_embed_input, 
                                                      sequence_length=source_sequence_length, dtype=tf.float32)
    
    return encoder_output, encoder_state

In [30]:
def process_decoder_input(data, vocab_to_int, batch_size):
    '''
    补充<GO>，并移除最后一个字符
    
    这里是为了构建 Decoder训练时的输入数据，使用target 而不是预测出的数据，提高精度
    '''
    # cut掉最后一个字符
    ending = tf.strided_slice(data, [0, 0], [batch_size, -1], [1, 1])
    decoder_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return decoder_input

In [31]:
def decoding_layer(target_letter_to_int, decoding_embedding_size, num_layers, rnn_size,
                   target_sequence_length, max_target_sequence_length, encoder_state, decoder_input,
                   encoder_outputs,source_sequence_length):
    '''
    构造Decoder层
    
    参数：
    - target_letter_to_int: target数据的映射表
    - decoding_embedding_size: embed向量大小
    - num_layers: 堆叠的RNN单元数量
    - rnn_size: RNN单元的隐层结点数量
    - target_sequence_length: target数据序列长度
    - max_target_sequence_length: target数据序列最大长度
    - encoder_state: encoder端编码的状态向量
    - decoder_input: decoder端输入
    
    - encoder_outputs :添加一个注意力机制
    - source_sequence_length 源数据长度
    '''
    
    '''
    # attention_states: [batch_size, max_time, num_units]
    attention_states = tf.transpose(encoder_outputs, [1, 0, 2])

    # Create an attention mechanism
    attention_mechanism = tf.contrib.seq2seq.LuongAttention(
        num_units, attention_states,
        memory_sequence_length=source_sequence_length)
    
    decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
    decoder_cell, attention_mechanism,
    attention_layer_size=num_units)
    
    
    '''
    # 1. Embedding
    target_vocab_size = len(target_letter_to_int)
    decoder_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    decoder_embed_input = tf.nn.embedding_lookup(decoder_embeddings, decoder_input)

    # 2. 构造Decoder中的RNN单元
    def get_decoder_cell(rnn_size):
        decoder_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return decoder_cell
    
    #2.1 添加注意力机制的RNN 单元
    def get_decoder_cell_attention(rnn_size):
         # attention_states: [batch_size, max_time, num_units]
#        attention_states = tf.transpose(encoder_outputs, [1, 0, 2])
        attention_states=encoder_outputs
         # Create an attention mechanism
        attention_mechanism = tf.contrib.seq2seq.LuongAttention(
        rnn_size, attention_states,
        memory_sequence_length=source_sequence_length)
        
        decoder_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
                            decoder_cell, attention_mechanism,
                            attention_layer_size=rnn_size)
        
        return decoder_cell
    
    
#     cell = tf.contrib.rnn.MultiRNNCell([get_decoder_cell(rnn_size) for _ in range(num_layers)])
    cell = tf.contrib.rnn.MultiRNNCell([get_decoder_cell_attention(rnn_size) for _ in range(num_layers)])

    
    
    
    # 3. Output全连接层
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))


    # 4. Training decoder
    with tf.variable_scope("decode"):
        # 得到help对象
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=decoder_embed_input,
                                                            sequence_length=target_sequence_length,
                                                            time_major=False)
        # 构造decoder
        training_decoder = tf.contrib.seq2seq.BasicDecoder(cell,
                                                           training_helper,
                                                           initial_state=cell.zero_state(dtype=tf.float32,batch_size=batch_size)
                                                        ,output_layer=output_layer) 

        training_decoder_output, _ ,_= tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                       impute_finished=True,
                                                                       maximum_iterations=max_target_sequence_length)
    # 5. Predicting decoder
    # 与training共享参数
    with tf.variable_scope("decode", reuse=True):
        # 创建一个常量tensor并复制为batch_size的大小
        start_tokens = tf.tile(tf.constant([target_letter_to_int['<GO>']], dtype=tf.int32), [batch_size], 
                               name='start_tokens')
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings,
                                                                start_tokens,
                                                                target_letter_to_int['<EOS>'])
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(cell,
                                                        predicting_helper,
                                                        initial_state=cell.zero_state(dtype=tf.float32,batch_size=batch_size)
                                                            ,output_layer=output_layer)
        predicting_decoder_output, _,_  = tf.contrib.seq2seq.dynamic_decode(predicting_decoder,
                                                            impute_finished=True,
                                                            maximum_iterations=max_target_sequence_length)
    
    return training_decoder_output, predicting_decoder_output

In [32]:
def seq2seq_model(input_data, targets, lr, target_sequence_length, 
                  max_target_sequence_length, source_sequence_length,
                  source_vocab_size, target_vocab_size,
                  encoder_embedding_size, decoder_embedding_size, 
                  rnn_size, num_layers):
    
    # 获取encoder的状态输出
    encoder_outputs, encoder_state = get_encoder_layer(input_data, 
                                  rnn_size, 
                                  num_layers, 
                                  source_sequence_length,
                                  source_vocab_size, 
                                  encoding_embedding_size)
    
    
    # 预处理后的decoder输入
    decoder_input = process_decoder_input(targets, target_letter_to_int, batch_size)
    
    # 将状态向量与输入传递给decoder
    training_decoder_output, predicting_decoder_output = decoding_layer(target_letter_to_int, 
                                                                       decoding_embedding_size, 
                                                                       num_layers, 
                                                                       rnn_size,
                                                                       target_sequence_length,
                                                                       max_target_sequence_length,
                                                                       encoder_state, 
                                                                       decoder_input,
                                                                        encoder_outputs,
                                                                        source_sequence_length
                                                                       ) 
    
    return training_decoder_output, predicting_decoder_output

In [33]:
# 超参数
# Number of Epochs
epochs =5
# Batch Size
batch_size =32
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.05

In [34]:
# 构造graph
train_graph = tf.Graph()

with train_graph.as_default():
    
    # 获得模型输入    
    input_data, targets, lr, target_sequence_length, max_target_sequence_length, source_sequence_length = get_inputs()
    
    training_decoder_output, predicting_decoder_output = seq2seq_model(input_data, 
                                                                      targets, 
                                                                      lr, 
                                                                      target_sequence_length, 
                                                                      max_target_sequence_length, 
                                                                      source_sequence_length,
                                                                      len(source_letter_to_int),
                                                                      len(target_letter_to_int),
                                                                      encoding_embedding_size, 
                                                                      decoding_embedding_size, 
                                                                      rnn_size, 
                                                                      num_layers)    
    
    training_logits = tf.identity(training_decoder_output.rnn_output, 'logits')
    predicting_logits = tf.identity(predicting_decoder_output.sample_id, name='predictions')
    
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

In [35]:
training_decoder_output

BasicDecoderOutput(rnn_output=<tf.Tensor 'decode/decoder/transpose:0' shape=(32, ?, 50000) dtype=float32>, sample_id=<tf.Tensor 'decode/decoder/transpose_1:0' shape=(32, ?) dtype=int32>)

In [36]:
def pad_sentence_batch(sentence_batch, pad_int):
    '''
    对batch中的序列进行补全，保证batch中的每行都有相同的sequence_length
    
    参数：
    - sentence batch
    - pad_int: <PAD>对应索引号
    '''
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [37]:
def get_batches(file_list,tokenize_path,batch_size,pad_int):
    '''
    定义生成器，用来获取tokenize下的所有content
    '''
    for item in file_list:
        source_path=os.path.join(tokenize_path,'content_'+item)
        target_path=os.path.join(tokenize_path,'title_'+item)
        with open(source_path,'r',encoding='utf-8')as sf:
            sources=[[int(word) for word in sentence.strip().split(' ')]for sentence in sf.readlines()]
        with open(target_path,'r',encoding='utf-8')as tf:
            targets=[[int(word) for word in sentence.strip().split(' ')]for sentence in tf.readlines()]
        
        for batch_i in range(0, len(sources)//batch_size):
            start_i = batch_i * batch_size
            sources_batch = sources[start_i:start_i + batch_size]
            targets_batch = targets[start_i:start_i + batch_size]
            # 补全序列
            pad_sources_batch = np.array(pad_sentence_batch(sources_batch, pad_int))
            pad_targets_batch = np.array(pad_sentence_batch(targets_batch, pad_int))

            # 记录每条记录的长度
            targets_lengths = []
            for target in targets_batch:
                targets_lengths.append(len(target))

            source_lengths = []
            for source in sources_batch:
                source_lengths.append(len(source))

            yield pad_targets_batch, pad_sources_batch, targets_lengths, source_lengths

In [38]:
data_path='./data_test'
file_list=os.listdir(data_path)
tokenize_path='./tokenize'
batch_size=32
pad_int=source_letter_to_int['<PAD>']

In [40]:
(valid_targets_batch, valid_sources_batch, valid_targets_lengths, valid_sources_lengths) = next(get_batches(file_list,tokenize_path,batch_size,pad_int))

In [41]:
display_step = 50 # 每隔50轮输出loss

# 查看是否有 checkpoint_f 这个文件，无则新建一个
if not os.path.exists('./checkpoint_f'):
        os.makedirs('./checkpoint_f')

checkpoint = "./checkpoint_f/trained_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
        
    for epoch_i in range(1, epochs+1):
        for batch_i, (targets_batch, sources_batch, targets_lengths, sources_lengths) in enumerate(
                get_batches(file_list,tokenize_path,batch_size,pad_int)):
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: sources_batch,
                 targets: targets_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths})

            if batch_i % display_step == 0:
            
                # 计算validation loss
                validation_loss = sess.run(
                [cost],
                {input_data: valid_sources_batch,
                 targets: valid_targets_batch,
                 lr: learning_rate,
                 target_sequence_length: valid_targets_lengths,
                 source_sequence_length: valid_sources_lengths})
                
                print('Epoch {:>3}/{} Batch {:>4}/{} - Training Loss: {:>6.3f}  - Validation loss: {:>6.3f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              '未知', 
                              loss, 
                              validation_loss[0]))

    
    
    # 保存模型
    saver = tf.train.Saver()
    saver.save(sess, checkpoint)
    print('Model Trained and Saved')

Epoch   1/5 Batch    0/未知 - Training Loss: 10.812  - Validation loss:  8.279
Epoch   1/5 Batch   50/未知 - Training Loss:  7.491  - Validation loss:  7.541
Epoch   1/5 Batch  100/未知 - Training Loss:  7.634  - Validation loss:  7.458
Epoch   1/5 Batch  150/未知 - Training Loss:  7.276  - Validation loss:  6.834
Epoch   1/5 Batch  200/未知 - Training Loss:  7.299  - Validation loss:  6.952
Epoch   2/5 Batch    0/未知 - Training Loss:  6.952  - Validation loss:  6.973
Epoch   2/5 Batch   50/未知 - Training Loss:  6.195  - Validation loss:  7.564
Epoch   2/5 Batch  100/未知 - Training Loss:  7.556  - Validation loss:  8.138
Epoch   2/5 Batch  150/未知 - Training Loss:  6.772  - Validation loss:  7.762
Epoch   2/5 Batch  200/未知 - Training Loss:  6.962  - Validation loss:  7.319
Epoch   3/5 Batch    0/未知 - Training Loss:  7.319  - Validation loss:  7.204
Epoch   3/5 Batch   50/未知 - Training Loss:  6.544  - Validation loss:  7.876
Epoch   3/5 Batch  100/未知 - Training Loss:  8.087  - Validation loss:  8.969

In [42]:
def source_to_seq(text):
    '''
    对源数据进行转换
    '''
    sequence_length = 120
    return [source_letter_to_int.get(word, source_letter_to_int['<UNK>']) for word in text.split(' ')] + [source_letter_to_int['<PAD>']]*(sequence_length-len(text))

In [44]:
# 输入一个单词
input_word = '''
 我 来说 两句 广州 男人 爱 拼 才会赢 无可否认 广州 男人 发了 广州 男人 财大气粗 似乎 分分秒秒 都 在 挣钱 广州 男人 形象 并 不 高大 皮肤 也 黝黑 但 他们 用 豪华 私家车 别墅 名牌 西服 名牌 衬衣 名牌 皮鞋 所 包装 出 的 气派 却是 令 许多 女人 心仪 不已 的 无牌 不 穿 无牌 不用 玩 的 就是 人民币 这 还 不 容易 吗 ? 广州 男人 有 一点点 坏 可话 又 说 回来 男人 不 坏 女人 不爱 嘛 而 被 天南地北 蜂涌
 '''
text = source_to_seq(input_word)

checkpoint = "./checkpoint_f/trained_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # 加载模型
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('inputs:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      target_sequence_length: [len(input_word)]*batch_size, 
                                      source_sequence_length: [len(input_word)]*batch_size})[0] 


pad = source_letter_to_int["<PAD>"] 

print('原始输入:', input_word)

print('\nSource')
print('  Word 编号:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([source_int_to_letter[i] for i in text])))

print('\nTarget')
print('  Word 编号:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([target_int_to_letter[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./checkpoint_f/trained_model.ckpt
原始输入: 
 我 来说 两句 广州 男人 爱 拼 才会赢 无可否认 广州 男人 发了 广州 男人 财大气粗 似乎 分分秒秒 都 在 挣钱 广州 男人 形象 并 不 高大 皮肤 也 黝黑 但 他们 用 豪华 私家车 别墅 名牌 西服 名牌 衬衣 名牌 皮鞋 所 包装 出 的 气派 却是 令 许多 女人 心仪 不已 的 无牌 不 穿 无牌 不用 玩 的 就是 人民币 这 还 不 容易 吗 ? 广州 男人 有 一点点 坏 可话 又 说 回来 男人 不 坏 女人 不爱 嘛 而 被 天南地北 蜂涌
 

Source
  Word 编号:    [1, 17, 168, 558, 593, 880, 960, 6032, 1, 47153, 593, 880, 15775, 593, 880, 28624, 1040, 47154, 22, 7, 11648, 593, 880, 1120, 49, 24, 6544, 1852, 13, 25494, 35, 51, 157, 2323, 4347, 760, 5522, 21054, 5522, 19374, 5522, 14891, 159, 4065, 201, 5, 17978, 2124, 1187, 526, 915, 16806, 6945, 5, 19375, 24, 1192, 19375, 1982, 2815, 5, 83, 430, 34, 44, 24, 825, 454, 39, 593, 880, 14, 8056, 7124, 1, 93, 33, 2248, 880, 24, 7124, 915, 21055, 6545, 36, 47, 1, 1, 4]
  Input Words: <UNK> 我 来说 两句 广州 男人 爱 拼 <UNK> 无可否认 广州 男人 发了 广州 男人 财大气粗 似乎 分分秒秒 都 在 挣钱 广州 男人 形象 并 不 高大 皮肤 也 黝黑 但 他们 用 豪华 私家车 别墅 名牌 西服 名牌 衬衣 名牌 皮鞋 所 包装 出 的 气派 却是 令 许多 女人 心仪 不已 的 无牌 不 穿 无牌 不用 玩 的 就是 